<a href="https://colab.research.google.com/github/mohanrajmit/AgeCalculation/blob/master/vgg16_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/mohanrajmit/AgeCalculation.git

fatal: destination path 'AgeCalculation' already exists and is not an empty directory.


Age Estimation for Indian Movie Face Dataset

In [2]:
# keras imports

from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input
from keras.applications import vgg16
from keras.applications.inception_resnet_v2 import InceptionResNetV2

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time
import pandas as pd




In [3]:
data=pd.read_csv("/content/AgeCalculation/dataset/train.csv")

image_name = data["ID"]
label_name = data["Class"]

features = []
labels = []

In [4]:
data.shape

(2605, 2)

In [5]:
pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
image_size = (224, 224)

In [6]:
dataset_path="/content/AgeCalculation/dataset/Train"

In [7]:
labels=[]
images_lst=[]
for image_name1,label in zip(image_name,label_name):	
	print("the image name  {} and label{}" .format(image_name1,label))
	image_path= str(dataset_path+"/"+image_name1)
	labels.append(label)	#print(image_path)


	img = image.load_img(image_path, target_size=image_size)
	x = image.img_to_array(img)
	images_lst.append(x)




the image name  377.jpg and labelMIDDLE
the image name  17814.jpg and labelYOUNG
the image name  21283.jpg and labelMIDDLE
the image name  16496.jpg and labelYOUNG
the image name  4487.jpg and labelMIDDLE
the image name  6283.jpg and labelMIDDLE
the image name  23495.jpg and labelYOUNG
the image name  7100.jpg and labelYOUNG
the image name  6028.jpg and labelYOUNG
the image name  22617.jpg and labelOLD
the image name  11177.jpg and labelYOUNG
the image name  2462.jpg and labelMIDDLE
the image name  24116.jpg and labelMIDDLE
the image name  17689.jpg and labelMIDDLE
the image name  728.jpg and labelMIDDLE
the image name  3003.jpg and labelMIDDLE
the image name  14408.jpg and labelOLD
the image name  6679.jpg and labelYOUNG
the image name  15152.jpg and labelOLD
the image name  24784.jpg and labelMIDDLE
the image name  9970.jpg and labelYOUNG
the image name  22550.jpg and labelOLD
the image name  150.jpg and labelYOUNG
the image name  7379.jpg and labelMIDDLE
the image name  15387.jpg an

In [8]:
# Create a single numpy array with all the images we loaded
x_train = np.array(images_lst)

# Also convert the labels to a numpy array
y_train = np.array(labels)

# Normalize image data to 0-to-1 range
x_train = vgg16.preprocess_input(x_train)

# Load a pre-trained neural network to use as a feature extractor
#model = ResNet50(weights='imagenet',include_top=False, input_shape=(224, 224, 3))
#pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Extract features for each image (all in one pass)
features_x = pretrained_nn.predict(x_train)




In [9]:
features_x.shape

(2605, 7, 7, 512)

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import joblib
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import confusion_matrix

In [11]:
# encode the labels using LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, 3)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(features_x, y_train, test_size=0.20, random_state=42)



In [15]:
X_train.shape

(2084, 7, 7, 512)

In [16]:
y_train[0]

array([1., 0., 0.], dtype=float32)

In [17]:
# Create a model and add layers
model = Sequential()

model.add(Flatten(input_shape=X_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=['accuracy']
)



In [ ]:
#class_names = ["not_glaucoma","glaucoma"]
# Train the model
H = model.fit(X_train,y_train,validation_data=(X_test, y_test),epochs=50,shuffle=True)

Epoch 1/50
66/66 [==============================] - 2s 9ms/step - loss: 8.0943 - accuracy: 0.5206 - val_loss: 0.9611 - val_accuracy: 0.6200
Epoch 2/50
66/66 [==============================] - 0s 6ms/step - loss: 0.8610 - accuracy: 0.6358 - val_loss: 0.8544 - val_accuracy: 0.5797
Epoch 3/50
66/66 [==============================] - 0s 6ms/step - loss: 0.6999 - accuracy: 0.6905 - val_loss: 0.8196 - val_accuracy: 0.6315
Epoch 4/50
66/66 [==============================] - 0s 6ms/step - loss: 0.6189 - accuracy: 0.7179 - val_loss: 0.8267 - val_accuracy: 0.6775
Epoch 5/50
66/66 [==============================] - 0s 6ms/step - loss: 0.5580 - accuracy: 0.7486 - val_loss: 0.7688 - val_accuracy: 0.6756
Epoch 6/50
66/66 [==============================] - 0s 6ms/step - loss: 0.4980 - accuracy: 0.7716 - val_loss: 0.8663 - val_accuracy: 0.6679
Epoch 7/50
66/66 [==============================] - 0s 6ms/step - loss: 0.4867 - accuracy: 0.7668 - val_loss: 0.8940 - val_accuracy: 0.6679
Epoch 8/50
66/66 [==

In [ ]:
# encode the labels using LabelEncoder
le = LabelEncoder()
le_labels = le.fit_transform(labels)

# get the shape of training labels
print ("[STATUS] training labels: {}".format(le_labels))
print ("[STATUS] training labels shape: {}".format(le_labels.shape))

# save features and labels
h5f_data = h5py.File("features.h5", 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File("label.h5", 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

h5f_data.close()
h5f_label.close()

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import numpy as np
import h5py
import os
import json
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
(trainData, testData, trainLabels, testLabels) = train_test_split(np.array(features),
                                                                  np.array(labels),
                                                                  test_size=test_size,
                                                                  random_state=seed)

print ("[INFO] splitted train and test data...")
print ("[INFO] train data  : {}".format(trainData.shape))
print ("[INFO] test data   : {}".format(testData.shape))
print ("[INFO] train labels: {}".format(trainLabels.shape))
print ("[INFO] test labels : {}".format(testLabels.shape))

# use logistic regression as the model
print ("[INFO] creating model...")
model = LogisticRegression(random_state=seed)
model.fit(trainData, trainLabels)

# use rank-1 and rank-5 predictions
print ("[INFO] evaluating model...")
f = open(results, "w")
rank_1 = 0
rank_5 = 0

# loop over test data
for (label, features) in zip(testLabels, testData):
	# predict the probability of each class label and
	# take the top-5 class labels
	predictions = model.predict_proba(np.atleast_2d(features))[0]
	predictions = np.argsort(predictions)[::-1][:5]

	# rank-1 prediction increment
	if label == predictions[0]:
		rank_1 += 1

	# rank-5 prediction increment
	if label in predictions:
		rank_5 += 1

# convert accuracies to percentages
rank_1 = (rank_1 / float(len(testLabels))) * 100
rank_5 = (rank_5 / float(len(testLabels))) * 100

# write the accuracies to file
f.write("Rank-1: {:.2f}%\n".format(rank_1))
f.write("Rank-5: {:.2f}%\n\n".format(rank_5))

# evaluate the model of test data
preds = model.predict(testData)

# write the classification report to file
f.write("{}\n".format(classification_report(testLabels, preds)))
f.close()

# dump classifier to file
print ("[INFO] saving model...")
pickle.dump(model, open(classifier_path, 'wb'))

# display the confusion matrix
print ("[INFO] confusion matrix")

# get the list of training lables
labels = sorted(list(os.listdir(train_path)))

# plot the confusion matrix
cm = confusion_matrix(testLabels, preds)
sns.heatmap(cm,
            annot=True,
            cmap="Set2")
plt.show()